### 약 이름 뽑아오기

# 메트릭스 저장하는 함수 만들기

In [3]:
import datetime
import pandas as pd

def save_df(df):
    now = datetime.datetime.now()
    format = "%m%d_%H%M"
    filename = now.strftime(format)
    df.to_csv("matrix/matrix_" + filename + ".csv")

import pandas as pd
test = pd.DataFrame()
save_df(test)

In [4]:
import datetime
import codecs
import urllib
import requests
import json
from bs4 import BeautifulSoup
import pandas as pd

def cur_timestamp(): # 현재 시간을 타임스탬프로 만들어주는 함수
    now = datetime.datetime.now()
    timestamp = int(now.timestamp()*1000)
    return str(timestamp)

def convert_to_utf8(string): # 한글을 url에 입력하게 변경해줌
    return urllib.parse.quote(string)

def get_response(url):
    headers = {
    'Accept': 'application/json, text/javascript, */*; q=0.01',
    'Accept-Encoding': 'gzip, deflate, br',
    'Accept-Language': 'ko-KR,ko;q=0.9,en-US;q=0.8,en;q=0.7',
    'Connection': 'keep-alive',
    'Cookie': 'ASPSESSIONIDQQCTSAQD=DPKPMKCCGAOKCEILBBICOKFF; _gid=GA1.2.68717777.1701659617; _ga_D05L6BYNX0=GS1.1.1701659616.1.1.1701660707.60.0.0; _ga=GA1.1.1789168813.1701659617',
    'Host': 'www.health.kr',
    'Referer': 'https://www.health.kr/interaction/food.asp',
    'Sec-Fetch-Dest': 'empty',
    'Sec-Fetch-Mode': 'cors',
    'Sec-Fetch-Site': 'same-origin',
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/119.0.0.0 Safari/537.36',
    'X-Requested-With': 'XMLHttpRequest',
    'sec-ch-ua': '"Google Chrome";v="119", "Chromium";v="119", "Not?A_Brand";v="24"',
    'sec-ch-ua-mobile': '?0',
    'sec-ch-ua-platform': '"Windows"'}
    response = requests.get(url, headers=headers)
    
    return json.loads(response.text)


def search_med(name):
    time = cur_timestamp()
    search_name = convert_to_utf8(name)
    url = f'https://www.health.kr/interaction/ajax/ajax_food_list.asp?search_word={search_name}&_={time}'
    try :
        resp = get_response(url)
        return pd.DataFrame(resp)
    except json.JSONDecodeError:
        # JSON 디코딩 오류 발생 시
        print(f"Error: {name}")
        return "Error"
# search_med('비아그라') - dcode, dname, dename, dtype, dsunb,info_count, ingr_mg

def search_ingd(dcode):
    # target : https://www.health.kr/interaction/ajax/ajax_food_ingd.asp?drug_code=A11ABBBBB2544&_=1701671995366'
    time = cur_timestamp()
    url = f'https://www.health.kr/interaction/ajax/ajax_food_ingd.asp?drug_code={dcode}&_={time}'
    resp = get_response(url)
    df = pd.DataFrame(resp)
    df['dcode'] = dcode
    return df
# search_ingd('A11ABBBBB2544') : ingd_code, ingd_name


def search_info(ingd_code):
    time = cur_timestamp()
    url = f'https://www.health.kr/interaction/ajax/ajax_food_sunblist.asp?icode={ingd_code}&_={time}'
    resp = get_response(url)
    df = pd.DataFrame(resp)
    df['ingd_code'] = ingd_code
    return df


### 약이름 목록 만들기

1. 비아로 시작하는 약이름 목록 추출 df1
2. 추출한 약이름에서 성분과 내용 뽑아서 합치기 df2

In [5]:
info_list = pd.DataFrame()
type(info_list) != str

True

In [6]:
### 1. 약이름 목록 추출
from tqdm import tqdm

def make_csv(key_list):
    med_list = pd.DataFrame()
    ingd_list = pd.DataFrame()
    info_list = pd.DataFrame()
    
    for i in key_list:
        print("key: " + f"{i}")
        result = search_med(i)
        if type(result) != str:
            df1 = result[['dcode', 'dname']].copy()
        else :
            df1 = pd.DataFrame(columns=['dcode', 'dname'])
        med_list = pd.concat([med_list, df1], ignore_index=True).drop_duplicates(subset='dcode', keep = 'first')
    med_list.to_csv('data/med_list.csv')
    print("약 데이터 생성 완료. 총: " + f"{med_list.shape[0]}" + "개")

    for idx, row in tqdm(med_list.iterrows(), total = med_list.shape[0]):
        df = search_ingd(row.dcode)
        ingd_list = pd.concat([ingd_list, df], ignore_index= True).drop_duplicates(subset = ['ingd_code', 'dcode'], keep = 'first')
    ingd_list.to_csv('data/ingd_list.csv')
    print("성분 데이터 생성 완료. 총 " + f"{ingd_list.shape[0]}" + "개")

    for idx, row in tqdm(ingd_list.iterrows(), total = ingd_list.shape[0]):
        df2 = search_info(row.ingd_code)
        info_list = pd.concat([info_list, df2], ignore_index= True).drop_duplicates(subset=['sunb_name', 'contents_origin', 'ingd_code'], keep = 'first')
    info_list.to_csv('data/info_list.csv')
    print("상호작용 데이터 생성 완료. 총 " + f"{info_list.shape[0]}개")   
   


In [8]:
search_list = ['가']
make_csv(search_list)

key: 가
약 데이터 생성 완료. 총: 2225개


100%|██████████| 2225/2225 [04:25<00:00,  8.38it/s]


성분 데이터 생성 완료. 총 8032개


 72%|███████▏  | 5821/8032 [18:36<07:04,  5.21it/s]   


ConnectTimeout: HTTPSConnectionPool(host='www.health.kr', port=443): Max retries exceeded with url: /interaction/ajax/ajax_food_sunblist.asp?icode=I005425&_=1702025806462 (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x000001EF291F0A90>, 'Connection to www.health.kr timed out. (connect timeout=None)'))

In [24]:
med_list = pd.read_csv('data/med_list.csv')
ingd_list = pd.read_csv('data/ingd_list.csv')
info_list = pd.read_csv('data/info_list.csv')

ingd_list[ingd_list.ingd_code == 'I003757']
med_list[med_list.dcode == 'A11APPPPP1623']

,Unnamed: 0,dcode,dname
315,315,A11APPPPP1623,건위황정


In [58]:
def load_relation(med_name):
    med_code = med_list[med_list.dname == med_name].dcode.iloc[0]
    ingd_df = ingd_list[ingd_list.dcode == med_code]
    result = []
    for idx, row in ingd_df.iterrows():
        result.append(info_list[info_list.ingd_code == row.ingd_code].contents_origin)
    return  result

load_relation('가나플럭스정20/1100mg') 

[9                          식전 30분~1시간에 복용하는 것이 가장 좋아요. 
 10    음주와 흡연을 삼가세요. 알코올과 니코틴은 위벽을 자극하여 위궤양, 가슴쓰림, 위산...
 11    {{카페인}} 복용시 주의하세요. 카페인은 하부식도괄약근을 느슨하게 하여 위산역류 ...
 12    마늘, 양파 등 자극성이 강한 식품은 위장장애를 유발하여 약효를 상쇄시킬 수 있으니...
 13     이 약은 비타민 B의 고갈을 일으킬 수 있어요. 보충제의 복용이 도움을 줄 수 있어요.
 14       이 약은 식품이나 칼슘 보충제로부터의 칼슘 흡수를 저해하므로 골다공증에 주의하세요.
 15                 캡슐을 열고 오렌지쥬스와 같은 산성쥬스와 함께 복용할 수 있어요.
 16                          캡슐을 개봉하여 약 알맹이를 씹어 먹으면 안돼요.
 Name: contents_origin, dtype: object,
 17                   공복(식사 1시간 이전 또는 식사 2시간 이후)에 복용하세요.
 18    음주와 흡연을 삼가세요. 알코올과 니코틴은 위벽을 자극하여 위궤양, 가슴쓰림, 위산...
 19    {{카페인}} 복용시 주의하세요. 카페인은 하부식도괄약근을 느슨하게 하여 위산역류 ...
 20    마늘, 양파 등 자극성이 강한 식품은 위장장애를 유발하여 약효를 상쇄시킬 수 있으니...
 21            철분 보충제를 복용시 이 약과 최소 2시간의 시간 간격을 두고 복용하세요.
 Name: contents_origin, dtype: object]

In [ ]:
# 약 10000개에 10분

In [14]:
df0 =pd.DataFrame()
df1 = search_med('가')
df2 = search_med('나')
(df1.shape, df2.shape)

((2231, 7), (8316, 7))

In [17]:
a = pd.concat([df0, df1], ignore_index=True).drop_duplicates(subset=['dcode'], keep = 'first')
b = pd.concat([a, df2], ignore_index=True).drop_duplicates(subset=['dcode'], keep = 'first')
a.shape, b.shape

((2231, 7), (10217, 7))

In [18]:
search_med('가')[['dcode', 'dname']]

,dcode,dname
0,A11AKP08C0036,10%포도당가엔에이케이주1
1,A11AKP08C0038,10%포도당가엔에이케이주2
2,A11APPPPP0629,10%포도당가엔에이케이주3
3,A11AOOOOO2364,5%포도당가엔에이케이주1
4,A11AOOOOO2365,5%포도당가엔에이케이주2(1000mL/백)
...,...,...
2226,2020110300013,히알본플러스원스주
2227,2020110300014,히알식스주
2228,2020110300015,히알탄원주
2229,2019071200008,힐스본정
